In [1]:
import cv2
import numpy as np
import time

# Brio camera setup
cameraNumber=0
fourcc = cv2.VideoWriter_fourcc(*'H264')   # 'M','J','P','G', *'XVID', *'H264'
cap = cv2.VideoCapture()
out=cv2.VideoWriter('output.mp4', fourcc, 20.0, (1500, 800))
cap.open(cameraNumber + 1 + cv2.CAP_DSHOW)

if not cap.isOpened():
    print("Error: Could not open Logitech Brio webcam.")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1500)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 800)
cap.set(cv2.CAP_PROP_FPS, 1)

cap.set(cv2.CAP_PROP_AUTOFOCUS,0)  # Turn off auto-focus
cap.set(cv2.CAP_PROP_FOCUS, 115) # Set the focus

# Detection
while True:
    ret, frame = cap.read()

    # Pre-processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9,9), 0)
    _, thresh = cv2.threshold(blurred, 160, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Circle detection using Hough Circle Transform
    circles = cv2.HoughCircles(
        gray, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=100, param2=30, minRadius=330, maxRadius=350
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Draw the outer circle
            cv2.circle(frame, (i[0], i[1]), i[2], (0, 255, 0), 2)
            cv2.circle(frame, (i[0], i[1]), 2, (0, 0, 255), 3)

            # Countours detection enclosed by circle
            count = 0
            for contour in contours:
                (x, y), radius = cv2.minEnclosingCircle(contour)
                center = (int(x), int(y))
                distance = np.sqrt((i[0] - center[0])**2 + (i[1] - center[1])**2)
                if distance <= i[2]:
                    count += 1
                    cv2.drawContours(frame, [contour], -1, (255, 0, 0), 2)

                # Display the count and condition result
                if count <= 30:
                   condition_result = "Good"
                else:
                   condition_result = "Not Good"
                        
            # Display the count
            cv2.putText(frame, f"Count: {count}", (i[0] - 20, i[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.putText(frame, f"Condition: {condition_result}", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 4, cv2.LINE_AA)

    # Display the frame
    cv2.imshow("Logitech Brio Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

threshold_value = 100
focus_value = 50

minRadius = 100
maxRadius = 150

# Callback function for trackbar
def update_threshold(val):
    global threshold_value
    threshold_value = val

# Callback function for the focus trackbar
def update_focus(val):
    global focus_value
    focus_value = val

# Callback function for the focus trackbar
def update_minRadius(val):
    global minRadius
    minRadius = val

# Callback function for the focus trackbar
def update_maxRadius(val):
    global maxRadius
    maxRadius = val

# Brio camera setup
cameraNumber=0
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')  # 'M','J','P','G', *'XVID', *'H264'
cap = cv2.VideoCapture()
cap.open(cameraNumber + 1 + cv2.CAP_DSHOW)

if not cap.isOpened():
    print("Error: Could not open Logitech Brio webcam.")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1400)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1200)
cap.set(cv2.CAP_PROP_FPS, 60)

cap.set(cv2.CAP_PROP_AUTOFOCUS,0)  # Turn off auto-focus
#cap.set(cv2.CAP_PROP_FOCUS, 115) # Set the focus

# Create a window and trackbar for thresholding
cv2.namedWindow('Video with Contours')
cv2.createTrackbar('Threshold', 'Video with Contours', threshold_value, 255, update_threshold)
cv2.createTrackbar('Focus', 'Video with Contours', focus_value, 255, update_focus)
cv2.createTrackbar('minRadius', 'Video with Contours', minRadius, 400, update_minRadius)
cv2.createTrackbar('maxRadius', 'Video with Contours', maxRadius, 600, update_minRadius)

# Detection
while True:
    ret, frame = cap.read()

    # Set the focus value from the trackbar
    cap.set(cv2.CAP_PROP_FOCUS, focus_value)

    # Pre-processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9,9), 0)

    # Get the threshold value from the trackbar
    threshold_value = cv2.getTrackbarPos('Threshold', 'Video with Contours')
    minRadius = cv2.getTrackbarPos('minRadius', 'Video with Contours')
    maxRadius = cv2.getTrackbarPos('maxRadius', 'Video with Contours')


    # Apply thresholding
    _, thresh = cv2.threshold(blurred, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Circle detection using Hough Circle Transform
    circles = cv2.HoughCircles(
        gray, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=100, param2=30, minRadius=minRadius, maxRadius=maxRadius
    )

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Draw the outer circle
            cv2.circle(frame, (i[0], i[1]), i[2], (0, 255, 0), 2)
            cv2.circle(frame, (i[0], i[1]), 2, (0, 0, 255), 3)

            # Countours detection enclosed by circle
            count = 0
            for contour in contours:
                (x, y), radius = cv2.minEnclosingCircle(contour)
                center = (int(x), int(y))
                distance = np.sqrt((i[0] - center[0])**2 + (i[1] - center[1])**2)
                if distance <= i[2]:
                    count += 1
                    cv2.drawContours(frame, [contour], -1, (255, 0, 0), 2)

                # Display the count and condition result
                if count <= 2:
                   condition_result = "Good"
                else:
                   condition_result = "Not Good"
                        
            # Display the count
            cv2.putText(frame, f"Count: {count}", (i[0] - 20, i[1] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.putText(frame, f"Condition: {condition_result}", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 4, cv2.LINE_AA) #(i[0] - 20, i[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
# Display the frame
    cv2.imshow("Video with Contours", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Release the video writer
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np

# Global variables for ROI parameters
roi_center = (700, 300)  # Example center of the circle (x, y)
roi_radius = 300  # Example radius

threshold_value = 100
focus_value = 50

# Callback function for the threshold trackbar
def update_threshold(val):
    global threshold_value
    threshold_value = val

# Callback function for the focus trackbar
def update_focus(val):
    global focus_value
    focus_value = val

# Callback function for the radius trackbar
def update_radius(val):
    global roi_radius
    roi_radius = val


# Brio camera setup
cameraNumber=0
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')  # 'M','J','P','G', *'XVID', *'H264'
cap = cv2.VideoCapture()
cap.open(cameraNumber + 1 + cv2.CAP_DSHOW)

if not cap.isOpened():
    print("Error: Could not open Logitech Brio webcam.")
    exit()

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1400)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1200)
cap.set(cv2.CAP_PROP_FPS, 60)

cap.set(cv2.CAP_PROP_AUTOFOCUS,0)  # Turn off auto-focus
#cap.set(cv2.CAP_PROP_FOCUS, 115) # Set the focus

# Create a window and trackbar for thresholding
cv2.namedWindow('Video with Contours')
cv2.createTrackbar('Threshold', 'Video with Contours', threshold_value, 255, update_threshold)
cv2.createTrackbar('Focus', 'Video with Contours', focus_value, 255, update_focus) 
cv2.createTrackbar('Radius', 'Video with Contours', roi_radius, 500, update_focus)

# Detection
while True:
    ret, frame = cap.read()

    # Set the focus value from the trackbar
    cap.set(cv2.CAP_PROP_FOCUS, focus_value)

    # Pre-processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9,9), 0)

    # Create a mask for the circular ROI
    mask = np.zeros_like(gray)
    cv2.circle(mask, roi_center, roi_radius, 255, 1)

    # Apply the mask to the blurred frame
    roi = cv2.bitwise_and(blurred, blurred, mask=mask)

    # Get the threshold value from the trackbar
    threshold_value = cv2.getTrackbarPos('Threshold', 'Video with Contours') 
    roi_radius = cv2.getTrackbarPos('Radius', 'Video with Contours')

    # Apply thresholding
    _, thresh = cv2.threshold(blurred, threshold_value, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the circular ROI
    cv2.circle(frame, roi_center, roi_radius, (0, 255, 0), 2)

    # Detect and draw the contours enclosed by the circular ROI
    enclosed_contours = 0  # Counter for enclosed contours
    for contour in contours:
        # Calculate the centroid of the contour
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])

            # Calculate the distance from the centroid to the ROI center
            distance = np.sqrt((cx - roi_center[0]) ** 2 + (cy - roi_center[1]) ** 2)

            # Draw the contour if it is within the circular ROI
            if distance <= roi_radius:
                cv2.drawContours(frame, [contour], -1, (255, 0, 0), 2)
                enclosed_contours += 1

    # Display the number of contours detected within the ROI
    #cv2.putText(frame, f"Contours: {enclosed_contours}", (100, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4, cv2.LINE_AA)

    # Display whether it's "Good" or "Bad" based on the number of enclosed contours
    condition_result = "Good" if enclosed_contours <= 1 else "Bad"
    cv2.putText(frame, f"Condition: {condition_result}", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4, cv2.LINE_AA)

    # Display the frame
    cv2.imshow("Video with Contours", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video writer
cap.release()
cv2.destroyAllWindows()